In [ ]:
import pandas as pd
import numpy as np
from IPython.display import Markdown
from plotly import graph_objects as go

from summer2 import CompartmentalModel, AgeStratification
from summer2.parameters import Parameter

from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_pop_death_data, get_death_rates
from tb_incubator.constants import compartments, latent_compartments, infectious_compartments, model_times, age_strata, indicator_names
from tb_incubator.model import build_model
from tb_incubator.utils import load_param_info, get_param_table

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

## Population data

We used estimates for the total population, deaths, and birth rate from the United Nations World Population Prospects [@unwpp2024].

In [ ]:
# Load age-stratified, population and death data
pop_death, description = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]
Markdown(description)

In [ ]:
# Load death rates
death_rates, description = get_death_rates()
Markdown(description)

## Model construction
This is arbitrary, but including some epidemiological transitions in the background.

In [ ]:
param_info = load_param_info()
params = param_info["value"]

model, desc = build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    params,
    model_times
)
Markdown(desc)

In [ ]:
# Track populations
age_pop_outputs = [model.request_output_for_compartments(s, compartments, strata={"age": str(s)}) for s in age_strata]

In [ ]:
model.run(params)

## Demographic results

In [ ]:
# Run and inspect results
model.run(params)
fig = model.get_derived_outputs_df()[[f"total_populationXage_{age}" for age in age_strata]].plot.area()
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))

In [ ]:
death_rates.plot()